### TO-DO
1. Add in matplotlib to side panel
2. Make fake graph with random sliders and boxes and wahtever to change the graph
3. Integrate it into the main editor instance
4. Connect internal model values with the sliders and graph and make it "real"

In [2]:
import jupyterlab_nodeeditor as jlne
import matplotlib